<a href="https://colab.research.google.com/github/ahmedabdulghany7/Competitive-Intelligence-Multi-Agent-System/blob/main/Event_Classification_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import json
import pandas as pd
from datetime import datetime
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, asdict
from enum import Enum
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class EventType(Enum):
    PRODUCT_LAUNCH = "product_launch"
    PRICING_CHANGES = "pricing_changes"
    MARKETING_CAMPAIGN = "marketing_campaign"
    EXPANSION = "expansion"
    UNKNOWN = "unknown"

@dataclass
class ClassificationResult:
    event_type: EventType
    confidence_score: float
    reasoning: str
    extracted_entities: Dict[str, List[str]]
    metadata: Dict[str, any]

class MobileCompanyEventClassifier:
    """
    Event Classification Agent for Mobile Phone Companies
    Designed to classify competitive intelligence signals into key event categories
    """

    def __init__(self):
        self.setup_patterns()
        self.setup_mobile_companies()

    def setup_patterns(self):
        """Define regex patterns and keywords for each event type"""

        # Product Launch patterns
        self.product_launch_patterns = {
            'keywords': [
                'launch', 'unveil', 'announce', 'introduce', 'reveal', 'debut',
                'new phone', 'new device', 'new model', 'flagship', 'series',
                'iphone', 'galaxy', 'pixel', 'xiaomi', 'oneplus', 'huawei',
                'smartphone', 'tablet', 'watch', 'earbuds', 'airpods'
            ],
            'phrases': [
                r'(launching|unveiling|announcing|introducing)\s+(new|latest)',
                r'(iphone|galaxy|pixel)\s+\d+',
                r'new\s+(flagship|device|phone|smartphone)',
                r'(coming|available)\s+(soon|next|this)',
                r'pre-?order\s+(now|available|starts)'
            ]
        }

        # Pricing Changes patterns
        self.pricing_patterns = {
            'keywords': [
                'price', 'cost', 'discount', 'sale', 'offer', 'deal', 'promotion',
                'cheaper', 'expensive', 'cut', 'reduce', 'increase', 'drop',
                '$', '€', '£', 'yuan', 'rupee', 'dollar', 'euro', 'pound'
            ],
            'phrases': [
                r'price\s+(cut|drop|reduction|increase|change)',
                r'(discount|sale|offer|deal)\s+on',
                r'\$\d+\s+(off|discount)',
                r'(starting|priced)\s+(at|from)\s+\$\d+',
                r'(more|less)\s+expensive',
                r'(black friday|cyber monday|holiday)\s+(sale|deal)'
            ]
        }

        # Marketing Campaign patterns
        self.marketing_patterns = {
            'keywords': [
                'campaign', 'advertisement', 'commercial', 'marketing', 'promo',
                'brand', 'ambassador', 'sponsor', 'partnership', 'collaboration',
                'social media', 'instagram', 'twitter', 'youtube', 'tiktok',
                'celebrity', 'influencer', 'endorsement'
            ],
            'phrases': [
                r'(marketing|advertising)\s+campaign',
                r'(brand|celebrity)\s+ambassador',
                r'(partnership|collaboration)\s+with',
                r'(social media|digital)\s+(campaign|marketing)',
                r'(commercial|ad)\s+(featuring|with)',
                r'(sponsor|endorsement)\s+(deal|agreement)'
            ]
        }

        # Expansion patterns
        self.expansion_patterns = {
            'keywords': [
                'expansion', 'expand', 'market', 'launch', 'enter', 'new market',
                'international', 'global', 'worldwide', 'country', 'region',
                'store', 'retail', 'opening', 'factory', 'manufacturing',
                'partnership', 'acquisition', 'merge', 'investment'
            ],
            'phrases': [
                r'(expand|expanding)\s+(to|into|in)',
                r'(launch|launching)\s+in\s+\w+',
                r'(new|first)\s+(store|retail|factory)',
                r'(enter|entering)\s+(the)?\s+\w+\s+market',
                r'(global|international)\s+(expansion|launch)',
                r'(partnership|deal)\s+in\s+\w+'
            ]
        }

    def setup_mobile_companies(self):
        """Define mobile phone companies and their variants"""
        self.mobile_companies = {
            'Apple': ['apple', 'iphone', 'ios', 'mac', 'ipad', 'airpods', 'apple watch'],
            'Samsung': ['samsung', 'galaxy', 'note', 'fold', 'flip', 'buds'],
            'Google': ['google', 'pixel', 'android', 'nexus'],
            'Xiaomi': ['xiaomi', 'mi', 'redmi', 'poco'],
            'OnePlus': ['oneplus', 'one plus', 'nord'],
            'Huawei': ['huawei', 'honor', 'mate', 'p series'],
            'Oppo': ['oppo', 'find', 'reno'],
            'Vivo': ['vivo', 'iqoo', 'nex'],
            'Sony': ['sony', 'xperia'],
            'Nokia': ['nokia', 'hmd global'],
            'Motorola': ['motorola', 'moto'],
            'LG': ['lg electronics', 'lg', 'wing']
        }

    def extract_entities(self, text: str) -> Dict[str, List[str]]:
        """Extract relevant entities from text"""
        entities = {
            'companies': [],
            'products': [],
            'financial_figures': [],
            'dates': [],
            'locations': []
        }

        text_lower = text.lower()

        # Extract companies
        for company, variants in self.mobile_companies.items():
            for variant in variants:
                if variant in text_lower:
                    entities['companies'].append(company)
                    break

        # Extract product models (iPhone 15, Galaxy S24, etc.)
        product_patterns = [
            r'(iphone)\s+(\d+\s*(pro|max|plus|mini)?)',
            r'(galaxy)\s+(s\d+|note\d+|fold\d*|flip\d*)',
            r'(pixel)\s+(\d+\s*(pro|xl)?)',
            r'(xiaomi|mi)\s+(\d+\s*(pro|ultra|lite)?)'
        ]

        for pattern in product_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    product = ' '.join([part for part in match if part]).strip()
                else:
                    product = match
                entities['products'].append(product)

        # Extract financial figures
        financial_patterns = [
            r'\$[\d,]+(?:\.\d{2})?',
            r'€[\d,]+(?:\.\d{2})?',
            r'£[\d,]+(?:\.\d{2})?',
            r'(\d+)\s*%\s*(off|discount|increase|decrease)',
            r'(\d+)\s*(million|billion)\s*(dollars|euros|pounds|yuan)'
        ]

        for pattern in financial_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            entities['financial_figures'].extend([match if isinstance(match, str) else ' '.join(match) for match in matches])

        # Extract dates (basic patterns)
        date_patterns = [
            r'(january|february|march|april|may|june|july|august|september|october|november|december)\s+\d{1,2}',
            r'\d{1,2}/\d{1,2}/\d{4}',
            r'(q[1-4])\s+\d{4}',
            r'(next|this)\s+(week|month|quarter|year)'
        ]

        for pattern in date_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            entities['dates'].extend(matches)

        # Extract locations/countries
        location_patterns = [
            r'(united states|usa|us|america)',
            r'(china|chinese|beijing|shanghai)',
            r'(india|indian|mumbai|delhi)',
            r'(europe|european|germany|france|uk|britain)',
            r'(japan|japanese|tokyo)',
            r'(south korea|korean|seoul)',
            r'(global|worldwide|international)'
        ]

        for pattern in location_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            entities['locations'].extend(matches)

        # Remove duplicates and empty strings
        for key in entities:
            entities[key] = list(set([item for item in entities[key] if item and item.strip()]))

        return entities

    def calculate_pattern_score(self, text: str, patterns: Dict) -> Tuple[float, List[str]]:
        """Calculate confidence score based on pattern matching"""
        text_lower = text.lower()
        matches = []
        keyword_score = 0
        phrase_score = 0

        # Check keywords
        for keyword in patterns['keywords']:
            if keyword in text_lower:
                keyword_score += 1
                matches.append(f"keyword: {keyword}")

        # Check phrases (regex patterns)
        for phrase_pattern in patterns['phrases']:
            if re.search(phrase_pattern, text, re.IGNORECASE):
                phrase_score += 2  # Phrases weighted more heavily
                matches.append(f"pattern: {phrase_pattern}")

        # Calculate normalized score (0-1)
        total_keywords = len(patterns['keywords'])
        total_phrases = len(patterns['phrases'])

        if total_keywords + total_phrases == 0:
            return 0.0, matches

        # Normalize scores
        keyword_norm = min(keyword_score / total_keywords, 1.0) if total_keywords > 0 else 0
        phrase_norm = min(phrase_score / (total_phrases * 2), 1.0) if total_phrases > 0 else 0

        # Weighted combination (phrases more important)
        final_score = (keyword_norm * 0.3) + (phrase_norm * 0.7)

        return final_score, matches

    def classify_event(self, text: str, metadata: Optional[Dict] = None) -> ClassificationResult:
        """
        Main classification method
        Args:
            text: Input text to classify
            metadata: Optional metadata (source, timestamp, etc.)
        """
        if metadata is None:
            metadata = {}

        # Calculate scores for each event type
        scores = {}
        all_matches = {}

        # Product Launch
        product_score, product_matches = self.calculate_pattern_score(text, self.product_launch_patterns)
        scores[EventType.PRODUCT_LAUNCH] = product_score
        all_matches[EventType.PRODUCT_LAUNCH] = product_matches

        # Pricing Changes
        pricing_score, pricing_matches = self.calculate_pattern_score(text, self.pricing_patterns)
        scores[EventType.PRICING_CHANGES] = pricing_score
        all_matches[EventType.PRICING_CHANGES] = pricing_matches

        # Marketing Campaign
        marketing_score, marketing_matches = self.calculate_pattern_score(text, self.marketing_patterns)
        scores[EventType.MARKETING_CAMPAIGN] = marketing_score
        all_matches[EventType.MARKETING_CAMPAIGN] = marketing_matches

        # Expansion
        expansion_score, expansion_matches = self.calculate_pattern_score(text, self.expansion_patterns)
        scores[EventType.EXPANSION] = expansion_score
        all_matches[EventType.EXPANSION] = expansion_matches

        # Find the highest scoring category
        max_score = max(scores.values())

        if max_score < 0.1:  # Very low confidence threshold
            predicted_type = EventType.UNKNOWN
            confidence = 0.0
            reasoning = "No clear patterns detected for any event type"
        else:
            predicted_type = max(scores, key=scores.get)
            confidence = max_score

            # Generate reasoning
            matching_patterns = all_matches[predicted_type]
            reasoning = f"Classified as {predicted_type.value} (confidence: {confidence:.2f}) based on patterns: {', '.join(matching_patterns[:3])}"

        # Extract entities
        entities = self.extract_entities(text)

        # Add classification timestamp
        metadata['classification_timestamp'] = datetime.now().isoformat()
        metadata['all_scores'] = {event_type.value: score for event_type, score in scores.items()}

        return ClassificationResult(
            event_type=predicted_type,
            confidence_score=confidence,
            reasoning=reasoning,
            extracted_entities=entities,
            metadata=metadata
        )

    def classify_batch(self, texts: List[str], metadata_list: Optional[List[Dict]] = None) -> List[ClassificationResult]:
        """Classify multiple texts at once"""
        if metadata_list is None:
            metadata_list = [{}] * len(texts)

        results = []
        for i, text in enumerate(texts):
            metadata = metadata_list[i] if i < len(metadata_list) else {}
            result = self.classify_event(text, metadata)
            results.append(result)

        return results

    def generate_report(self, results: List[ClassificationResult]) -> Dict:
        """Generate a summary report from classification results"""
        if not results:
            return {"error": "No results to analyze"}

        # Count events by type
        event_counts = {}
        for result in results:
            event_type = result.event_type.value
            event_counts[event_type] = event_counts.get(event_type, 0) + 1

        # Calculate average confidence by event type
        confidence_by_type = {}
        for event_type in event_counts:
            confidences = [r.confidence_score for r in results if r.event_type.value == event_type]
            confidence_by_type[event_type] = sum(confidences) / len(confidences) if confidences else 0

        # Extract most mentioned companies
        all_companies = []
        for result in results:
            all_companies.extend(result.extracted_entities.get('companies', []))

        company_counts = {}
        for company in all_companies:
            company_counts[company] = company_counts.get(company, 0) + 1

        top_companies = sorted(company_counts.items(), key=lambda x: x[1], reverse=True)[:5]

        # High confidence results
        high_confidence_results = [r for r in results if r.confidence_score >= 0.7]

        return {
            "total_events": len(results),
            "event_distribution": event_counts,
            "average_confidence_by_type": confidence_by_type,
            "top_mentioned_companies": dict(top_companies),
            "high_confidence_events": len(high_confidence_results),
            "classification_accuracy_estimate": len(high_confidence_results) / len(results) if results else 0
        }

# Example usage and testing
def run_example_classification():
    """Run example classifications to demonstrate the system"""

    # Initialize classifier
    classifier = MobileCompanyEventClassifier()

    # Sample mobile industry news/signals
    test_signals = [
        "Apple unveils new iPhone 15 Pro with titanium design starting at $999",
        "Samsung Galaxy S24 series price drops by 20% ahead of Black Friday sales",
        "Google launches massive Pixel marketing campaign featuring celebrities",
        "Xiaomi expands manufacturing operations to India with new $500M factory",
        "OnePlus announces partnership with Hasselblad for camera technology",
        "Apple reports strong iPhone sales in Q4 2024 earnings call",
        "Samsung Galaxy Fold 5 now available for pre-order at $1799",
        "Google's new Pixel advertising campaign goes viral on TikTok",
        "Huawei opens first flagship store in Paris, expanding European presence",
        "iPhone 14 gets permanent price cut to $699 following iPhone 15 launch"
    ]

    print("🤖 Mobile Company Event Classification Agent")
    print("=" * 50)

    # Classify each signal
    results = []
    for i, signal in enumerate(test_signals, 1):
        print(f"\n📱 Signal {i}: {signal[:60]}...")

        result = classifier.classify_event(signal, {'source': 'test_data', 'signal_id': i})
        results.append(result)

        print(f"   📊 Event Type: {result.event_type.value}")
        print(f"   🎯 Confidence: {result.confidence_score:.2f}")
        print(f"   🏢 Companies: {', '.join(result.extracted_entities.get('companies', ['None']))}")
        print(f"   📱 Products: {', '.join(result.extracted_entities.get('products', ['None']))}")
        print(f"   💰 Financial: {', '.join(result.extracted_entities.get('financial_figures', ['None']))}")

    # Generate summary report
    print("\n" + "=" * 50)
    print("📈 CLASSIFICATION SUMMARY REPORT")
    print("=" * 50)

    report = classifier.generate_report(results)

    print(f"Total Events Processed: {report['total_events']}")
    print("\n📊 Event Distribution:")
    for event_type, count in report['event_distribution'].items():
        print(f"   • {event_type.replace('_', ' ').title()}: {count}")

    print(f"\n🎯 High Confidence Events: {report['high_confidence_events']}")
    print(f"📈 Estimated Accuracy: {report['classification_accuracy_estimate']:.1%}")

    print("\n🏢 Top Mentioned Companies:")
    for company, mentions in report['top_mentioned_companies'].items():
        print(f"   • {company}: {mentions} mentions")

    return classifier, results

# Google Colab specific setup
def setup_for_colab():
    """Setup function for Google Colab environment"""
    print("🔧 Setting up Event Classification Agent for Google Colab...")

    # Install required packages if not available
    try:
        import pandas as pd
    except ImportError:
        print("Installing pandas...")
        !pip install pandas

    print("✅ Setup complete! Ready to classify mobile industry events.")

    # Run example
    classifier, results = run_example_classification()

    return classifier, results

if __name__ == "__main__":
    # Run setup and examples
    classifier, results = setup_for_colab()

🔧 Setting up Event Classification Agent for Google Colab...
✅ Setup complete! Ready to classify mobile industry events.
🤖 Mobile Company Event Classification Agent

📱 Signal 1: Apple unveils new iPhone 15 Pro with titanium design startin...
   📊 Event Type: product_launch
   🎯 Confidence: 0.17
   🏢 Companies: Apple
   📱 Products: iPhone 15 Pro Pro
   💰 Financial: $999

📱 Signal 2: Samsung Galaxy S24 series price drops by 20% ahead of Black ...
   📊 Event Type: pricing_changes
   🎯 Confidence: 0.28
   🏢 Companies: Samsung
   📱 Products: Galaxy S24
   💰 Financial: 

📱 Signal 3: Google launches massive Pixel marketing campaign featuring c...
   📊 Event Type: marketing_campaign
   🎯 Confidence: 0.15
   🏢 Companies: Google
   📱 Products: 
   💰 Financial: 

📱 Signal 4: Xiaomi expands manufacturing operations to India with new $5...
   📊 Event Type: unknown
   🎯 Confidence: 0.00
   🏢 Companies: Xiaomi
   📱 Products: 
   💰 Financial: $500

📱 Signal 5: OnePlus announces partnership with Hasselb